In [ ]:
!pip install telethon requests qrcode Pillow nest_asyncio opencv-python

In [ ]:
# Установка необходимых библиотек
!pip install telethon requests nest_asyncio

# Импорт библиотек
import os
import json
import asyncio
from telethon import TelegramClient, functions
from telethon.tl.types import MessageMediaDocument
import nest_asyncio

# Применение nest_asyncio для корректной работы
nest_asyncio.apply()

# --- Ваши данные из GitHub Secrets ---
# API_ID преобразуем в число, остальные - строки
api_id = int(os.environ['API_ID'])
api_hash = os.environ['API_HASH']
phone = os.environ['PHONE_NUMBER']
# Переменная n8n_webhook_url теперь используется только в main.yml
# ---------------------------------------------

# --- Список каналов для парсинга ---
# Используйте ТОЛЬКО короткое имя канала (username) с @
channel_names = ['@sochitypical', '@officialsochi', '@soch01','@sochi_news4', '@agproshunin', '@livesochi', '@sochi03', '@sochi_online', '@overheardsochi', '@officialsochi']
# ---------------------------------------------

async def main():
    # Подключение к Telegram
    client = TelegramClient('my_session', api_id, api_hash)
    await client.connect()

    # Проверка авторизации
    if not await client.is_user_authorized():
        print("Ошибка: не удалось авторизоваться. Убедитесь, что файл my_session.session загружен в репозиторий.")
        # Завершаем выполнение, если нет авторизации
        return

    # --- Попытка присоединиться для получения entity ---
    for channel_name in channel_names:
        try:
            await client(functions.channels.JoinChannelRequest(channel_name))
            print(f"Успешно присоединились или проверили доступ к: {channel_name}")
        except Exception as e:
            print(f"Не удалось присоединиться или проверить доступ к {channel_name}: {e}")
    # -------------------------------------------------------------

    # Получение последних 5 сообщений из каждого канала в списке
    results = []
    for channel_name in channel_names:
        print(f"Попытка парсинга канала: {channel_name}")
        try:
            async for msg in client.iter_messages(channel_name, limit=5):
                # Проверяем, есть ли в сообщении хоть что-то (текст ИЛИ медиа)
                if msg.text or msg.media:

                    # Создаем базовый объект для сообщения
                    message_data = {
                        'channel': channel_name,
                        'message_id': msg.id,
                        'text': msg.text if msg.text else '',
                        'date': str(msg.date),
                        'has_media': False,
                        'media_type': None,
                        'file_name': None,
                    }

                    # Добавляем информацию о медиа
                    if msg.media:
                        message_data['has_media'] = True
                        message_data['media_type'] = type(msg.media).__name__

                        # Пытаемся получить имя файла
                        if isinstance(msg.media, MessageMediaDocument) and msg.media.document.attributes:
                            for attr in msg.media.document.attributes:
                                if hasattr(attr, 'file_name'):
                                    message_data['file_name'] = attr.file_name
                                    break

                    results.append(message_data)

        except Exception as e:
            # КРИТИЧЕСКАЯ ОШИБКА. Выводим ее в лог, чтобы понять причину пропуска.
            print(f"Критическая ошибка при парсинге канала {channel_name}: {e}")
            continue

    # ------------------------------------------------------------------
    # НОВЫЙ БЛОК: Сохранение данных в локальный файл
    # ------------------------------------------------------------------
    OUTPUT_FILE = 'parsed_messages.json'

    try:
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            # requests больше не нужен, но json нужен для записи в файл
            json.dump(results, f, ensure_ascii=False, indent=4)
        print(f"✅ Данные успешно сохранены в файл: {OUTPUT_FILE}. Количество сообщений: {len(results)}")
    except Exception as e:
        print(f"🛑 Ошибка при сохранении данных в файл: {e}")
        
    # ------------------------------------------------------------------

    # Отключение от Telegram
    await client.disconnect()

if __name__ == '__main__':
    # Добавляем try/except для отладки в случае ошибки asyncio
    try:
        asyncio.run(main())
    except Exception as e:
        print(f"Критическая ошибка выполнения main: {e}")